In [4]:
%run -i writefile2.py

In [10]:
%%writefile2 --name server.py

from flask import Flask, render_template, send_from_directory
from flask_socketio import SocketIO
from io import StringIO
import os
import pandas as pd
import require
import functools
get_data_for_country = require.single( "get_data_for_country" )
categorical_to_dummy = require.single( "categorical_to_dummy" )
crop_to_valid_range = require.single( "crop_to_valid_range" )

app = Flask( __name__, static_folder = "static", static_url_path = '' )
socketio = SocketIO(app)

@functools.cache
def load_data( country ):

    log( f"loading data for { country }" )
    return get_data_for_country( country, verbose = False, categorical_as_dummy = False )

@app.route('/')
def serve_index():
    return send_from_directory('static', 'index.html')

@socketio.event
def my_event(json):
    log('received json: ' + str(json))
    return "OK", 200

@socketio.event
def get_outcome_columns( ):
    
    return ['new_cases_smoothed_per_million', 'new_deaths_smoothed_per_million', 'weekly_hosp_admissions_per_million', ]

@socketio.event
def get_measure_columns( ):
    
    return ['new_vaccinations_smoothed_per_million',
       'new_tests_smoothed_per_thousand', 'c6m_stay_at_home_requirements',
       'c8ev_internationaltravel', 'h6m_facial_coverings',
       'c4m_restrictions_on_gatherings']

@functools.cache
def read_legend( column ):

    with open( f'legend/{ column }', "r" ) as file: 
        
        text = file.read( )

    data = {}
    lines = text.strip().split('\n')
    for line in lines:
        if '-' in line:
            key, value = line.split('-', 1)  # 1 is the maxsplit parameter.
            data[key.strip()] = value.strip()
    return data
    

@socketio.event
def get_columns( ):

    df = load_data( "Germany" )
    return [ dict( 
        name = c, 
        is_measure = i >= 3,
        is_categorical = i >= 3 and not c.startswith( "new" ),
        n_categories = [ 3, 4, 4, 4 ][ i - 5 ] if i >= 3 and not c.startswith( "new" ) else 0,
        legend = read_legend( c ) if i >= 3 and not c.startswith( "new" ) else { }) for i, c in enumerate( df.columns )]
        

@socketio.event
def get_countries( ):
    
    return [ "Germany", "Switzerland", "Italy", "France", "Belgium", "United States", "Spain", "United Kingdom", "Malaysia", "South Korea", "Chile" ]

@socketio.event
def get_data( country ):

    return df_to_csvs( load_data( country ))

def df_to_csvs( df ):

    def to_csv( s ):

        s.name = "close"
        return s.to_csv( )

    return { c: to_csv( df[ c ]) for c in df.columns }

def csvs_to_df( csvs ):

    df = pd.DataFrame( )
    for col in csvs:

        parsed = pd.read_csv(StringIO(csvs[ col ]), index_col = "date", parse_dates = True, low_memory = False )
        df[ col ] = parsed[ "close" ]
        df.index = parsed.index

    return df

@functools.cache
def train_model( method ):

    if method == "belief_ensemble":

        train_ensemble_node = require.single( "train_ensemble" )
        return train_ensemble_node.get_result( )

def check_df( df ):

    reference = categorical_to_dummy( load_data( 'Germany' ))
    assert df.columns.tolist( ) == reference.columns.tolist( )

@socketio.event
def predict( csvs, country, method = "belief_ensemble" ):

    model = train_model( method )
    df = crop_to_valid_range( categorical_to_dummy( csvs_to_df( csvs )), verbose = False )
    check_df( df )
    log( "making predictions..." )
    df_pred = model.predict_replace( df )
    log( "done" )
    return df_to_csvs( df_pred )

def log( * args ):

    print( * args )
    socketio.emit( "log", * args )
    socketio.sleep( 0 ) #force immediate send

def main( ):

    # preheat
    train_model( "belief_ensemble" )
    
    try:

        log( "running..." )
        log( "http://localhost:8080" )
        socketio.run( app, port = 8080, host = "0.0.0.0" )    

    except:
        
        print( "done." )

if "is_server" in os.environ:

    main( )

In [7]:
model = train_model( "belief_ensemble" )
print( "trained" )
predict( get_data( "Germany" ), "Germany" )

trained
making predictions...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.36it/s]
lhs shape = (1846, 1022)
lhs rank = 924
degrees of freedom = 1022
done


{'new_cases_smoothed_per_million': 'date,close\n2020-03-01,\n2020-03-02,\n2020-03-03,\n2020-03-04,\n2020-03-05,\n2020-03-06,\n2020-03-07,\n2020-03-08,\n2020-03-09,\n2020-03-10,\n2020-03-11,\n2020-03-12,\n2020-03-13,\n2020-03-14,\n2020-03-15,\n2020-03-16,\n2020-03-17,\n2020-03-18,\n2020-03-19,\n2020-03-20,\n2020-03-21,\n2020-03-22,\n2020-03-23,\n2020-03-24,\n2020-03-25,\n2020-03-26,\n2020-03-27,\n2020-03-28,\n2020-03-29,\n2020-03-30,\n2020-03-31,\n2020-04-01,\n2020-04-02,\n2020-04-03,\n2020-04-04,\n2020-04-05,\n2020-04-06,\n2020-04-07,\n2020-04-08,\n2020-04-09,\n2020-04-10,\n2020-04-11,\n2020-04-12,\n2020-04-13,\n2020-04-14,\n2020-04-15,\n2020-04-16,\n2020-04-17,\n2020-04-18,\n2020-04-19,\n2020-04-20,\n2020-04-21,\n2020-04-22,\n2020-04-23,\n2020-04-24,\n2020-04-25,\n2020-04-26,\n2020-04-27,\n2020-04-28,\n2020-04-29,\n2020-04-30,\n2020-05-01,\n2020-05-02,\n2020-05-03,\n2020-05-04,\n2020-05-05,\n2020-05-06,\n2020-05-07,\n2020-05-08,\n2020-05-09,\n2020-05-10,\n2020-05-11,\n2020-05-12,\n202